<a href="https://colab.research.google.com/github/hscrown/paper_reading/blob/main/AlexNet_%ED%85%90%EC%84%9C%ED%94%8C%EB%A1%9C%EB%A1%9C_%EA%B5%AC%ED%98%84%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

AlexNet

In [1]:
# 데이터 전처리
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# CIFAR-100 데이터셋 로드 및 스플릿
(train_X, train_y), (test_X, test_y) = datasets.cifar100.load_data()

# 데이터 정규화 (픽셀 값 범위를 [0, 1]로 변환)
train_X = train_X / 255.0
test_X = test_X / 255.0

# CIFAR-100을 AlexNet 입력 크기인 224x224로 리사이즈
train_X = tf.image.resize(train_X, [224, 224])
test_X = tf.image.resize(test_X, [224, 224])

# 데이터셋 크기 확인
print("Train data shape:", train_X.shape)  # (50000, 224, 224, 3)
print("Test data shape:", test_X.shape)    # (10000, 224, 224, 3)

In [ ]:
# AlexNet 모델 정의 (논문 구조에 맞게 LRN과 드롭아웃 포함)
model = tf.keras.Sequential([
    # 첫 번째 Conv 레이어, 패딩:valid 패딩 없이 계산하여 특징 맵의 크기를 줄임
    layers.Conv2D(filters=96, kernel_size=(11, 11), strides=(4, 4), padding='valid', activation='relu',
                  input_shape=(224, 224, 3)),

    # 첫 번째 LRN 레이어
    layers.Lambda(lambda x: tf.nn.local_response_normalization(x, depth_radius=5, bias=1, alpha=1e-4, beta=0.75)),

    # 첫 번째 MaxPooling 레이어
    layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2)),

    # 두 번째 Conv 레이어 패딩:same 입력과 출력의 크기가 같도록 제로패딩추가
    layers.Conv2D(filters=256, kernel_size=(5, 5), strides=(1, 1), padding='same', activation='relu'),

    # 두 번째 LRN 레이어
    layers.Lambda(lambda x: tf.nn.local_response_normalization(x, depth_radius=5, bias=1, alpha=1e-4, beta=0.75)),

    # 두 번째 MaxPooling 레이어
    layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2)),

    # 세 번째 Conv 레이어
    layers.Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu'),

    # 네 번째 Conv 레이어
    layers.Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu'),

    # 다섯 번째 Conv 레이어
    layers.Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu'),

    # 세 번째 MaxPooling 레이어
    layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2)),

    # Flatten 레이어: 다차원 출력을 1차원으로 펼쳐서 Fully Connected 레이어에 전달
    layers.Flatten(),

    # 첫 번째 Fully Connected 레이어
    layers.Dense(units=4096, activation='relu'),
    # 첫 번째 드롭아웃 레이어
    # - 비율: 0.5, 첫 번째 Fully Connected 레이어 뒤에 추가
    layers.Dropout(0.5),

    # 두 번째 Fully Connected 레이어
    layers.Dense(units=4096, activation='relu'),
    # 두 번째 드롭아웃 레이어
    # - 비율: 0.5, 두 번째 Fully Connected 레이어 뒤에 추가
    layers.Dropout(0.5),

    # 출력 레이어: CIFAR-100에 맞춰 100개의 클래스로 분류
    layers.Dense(units=100, activation='softmax')
])

# 모델 컴파일
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 모델 요약 정보 출력
model.summary()

In [ ]:
# 모델 학습
history = model.fit(train_X, train_y, epochs=2, validation_split=0.2)

In [ ]:
# 결과 시각화
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], 'b-', label='Training Loss')
plt.plot(history.history['val_loss'], 'r--', label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], 'g-', label='Training Accuracy')
plt.plot(history.history['val_accuracy'], 'r--', label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# 모델 성능 평가
test_loss, test_accuracy = model.evaluate(test_X, test_y, verbose=0)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")